In [13]:
from graphics import *
import numpy as np

In [14]:
R=64
MAX_X = 1000
MAX_Y = 500
NODE_SIZE = 5
N_NODES =16

In [15]:
class Tree():
    def __init__(self):
        self.root = Node('root',R, Circle(Point(0.5*MAX_X/5, MAX_Y/2), NODE_SIZE))
    def add_child(self, name, node):
        n = self.root
        queue = []
        while n.name != name:
            queue.extend(n.children)
            n = queue.pop()
        n.children.append(node)
    
    
class Node():
    def __init__(self, name, cost, circle):
        self.name = name
        self.cost = cost
        self.circle = circle
        self.children = []

In [16]:
def get_node_budgets(n_nodes, uniform=False):
    if uniform:
        return [64 for i in range(n_nodes)]
    def get_mid(n_nodes, size):
        return range(int((n_nodes - size) /2), int((n_nodes + size)/2))
    budget = np.ones(n_nodes)
    for i in range(3):
        alloc = get_mid(n_nodes,64/(4**(i+1)))
        for n in alloc:
            budget[n] = 4**(i+1)
    return budget
        
    

In [17]:
tree = Tree()

for c in range(4):
    # Add Preprocessing Nodes
    node_name = 'layer1-'+str(c)
    circle = Circle(Point(1.5*MAX_X/5, (c+1)*MAX_Y/5), NODE_SIZE)
    tree.add_child('root', Node(node_name, R, circle))
    for m in range(4):
        model_num = (c*4)+m+1
        circle = Circle(Point(2.5*MAX_X/5, model_num/(N_NODES+1)* MAX_Y), NODE_SIZE/2)
        tree.add_child(node_name, Node('layer2-'+str(model_num), R/N_NODES, circle))
        
    

In [18]:
win = GraphWin('DAG', MAX_X, MAX_Y)
queue = [tree.root]
tree.root.circle.draw(win)
node_budget = get_node_budgets(16)
while len(queue):
    n = queue.pop()
    queue.extend(n.children)
    for c in n.children:
        c.circle.draw(win)
        line = Line(n.circle.getCenter(), c.circle.getCenter())
        line.draw(win)
        queue.extend(c.children)
        if 'layer2' in c.name:
            node_num = int(c.name.split('-')[1])-1
            line = Line(c.circle.getCenter(), Point(c.circle.getCenter().getX()+(2*node_budget[node_num]/64)*MAX_X/5+NODE_SIZE/2, c.circle.getCenter().getY()))
            line.draw(win)
win.postscript(file="dag.eps", colormode='color')
win.close()